# Анализ базы данных сервиса для чтения книг


Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, 
не посещают кафе и торговые центры. Зато стало больше времени для книг. 

Цель — проанализировать базу данных крупного сервиса для чтения книг по подписке. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Описание данных**

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

**Задания **

- Посчитать, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц;
- Определить автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками;
- Посчитайть среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

## Загрузка и изучение данных 

In [74]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [75]:
query = '''SELECT * FROM authors LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [76]:
query = '''SELECT * FROM publishers LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [77]:
query = '''SELECT * FROM ratings LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [78]:
query = '''SELECT * FROM reviews LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Сколько книг вышло после 1 января 2000 года

In [79]:
query = """SELECT COUNT(book_id) 
           FROM books
           WHERE publication_date > '2000-01-01'"""

con = engine.connect()

pd.io.sql.read_sql(sql=text(query), con=con)


,count
0,819


819 книг вышло после 1 января 2000 года.

## Количество обзоров и средняя оценка

In [80]:
query = """SELECT DISTINCT b.title,
                  b.book_id,
                  COUNT(review_id) OVER (PARTITION BY book_id) as quantity,
                  ROUND(AVG(rating) OVER (PARTITION BY book_id), 2) as avg_rating
           FROM books as b
           LEFT JOIN ratings as r USING(book_id)
           LEFT JOIN reviews as rw USING(book_id)"""

con = engine.connect()

pd.io.sql.read_sql(sql=text(query), con=con)


,title,book_id,quantity,avg_rating
0,The Devil and Miss Prym (On the Seventh Day #3),701,4,3.5
1,Twenty Thousand Leagues Under the Sea (Extraor...,947,15,4.6
2,Exodus,231,6,4.0
3,Vanity Fair,956,12,4.0
4,Mercy,429,4,3.5
...,...,...,...,...
995,A Passage to India,39,4,3.5
996,Swann's Way (In Search of Lost Time #1),609,4,3.0
997,Howl's Moving Castle (Howl's Moving Castle #1),334,24,4.5
998,Lasher (Lives of the Mayfair Witches #2),383,4,4.5


В результате мы вывели таблицу, в которой можно увидеть количество обзоров и среднюю оценку по каждой книге.

## Издательство, которое выпустило наибольшее число книг толще 50 страниц

In [86]:
query = """SELECT DISTINCT publisher,
                  COUNT(book_id) OVER (PARTITION BY publisher_id) as quantity
           FROM books as b
           LEFT JOIN publishers as p USING(publisher_id)
           WHERE b.num_pages > 50
           ORDER BY quantity DESC
           LIMIT 1"""

con = engine.connect()

pd.io.sql.read_sql(sql=text(query), con=con)

,publisher,quantity
0,Penguin Books,42


Penguin Books выпустило наибольшее количество книг - 42.

## Автор с самой высокой средней оценкой книг (50 и более оценок)

In [184]:
query = """WITH temp AS (SELECT *,
                  COUNT(rating_id) OVER (PARTITION BY book_id) as quantity
           FROM ratings)
           SELECT author,
                  ROUND(AVG(rating) OVER (PARTITION BY author_id), 1) as avg_rating
           FROM temp
           JOIN books as b USING(book_id)
           JOIN author as a USING(author_id)
           WHERE quantity >= 50
           ORDER BY avg_rating DESC
           LIMIT 1"""

con = engine.connect()

pd.io.sql.read_sql(sql=text(query), con=con)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.3


Наибольший рейтинг у книги J.K. Rowling совместно с Mary GrandPré - 4,3 балла. 

## Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [181]:
query = """SELECT AVG(q_reviews)
           FROM (SELECT username, COUNT(*) q_reviews
                 FROM reviews
                 WHERE username IN (SELECT username
                                    FROM ratings
                                    GROUP BY username
                                    HAVING COUNT(*) > 48)
           GROUP BY username) r"""

con = engine.connect()

pd.io.sql.read_sql(sql=text(query), con=con)

,avg
0,24.0


24 - среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

## Вывод

В результате исследования мы выяснили:
   - 819 книг вышло после 1 января 2000 года.
   - Penguin Books выпустило наибольшее количество книг - 42.
   - Наибольший рейтинг у книги J.K. Rowling совместно с Mary GrandPré - 4,3 балла.
   - 24 - среднее количество обзоров от пользователей, которые поставили больше 48 оценок.
Также мы дополнили таблицу количеством обзоров и средней оценкой по каждой книге.